In [66]:
from pyspark.sql import SparkSession

In [67]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [68]:
from pyspark.ml.regression import LinearRegression

In [69]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

23/08/06 18:24:03 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [70]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [71]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [72]:
lrModel = lr.fit(training)

23/08/06 18:24:03 WARN Instrumentation: [87bf7f23] regParam is zero, which might cause numerical instability and overfitting.


In [73]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [74]:
lrModel.intercept

0.14228558260358093

In [75]:
training_summary = lrModel.summary

In [76]:
training_summary.rootMeanSquaredError

10.16309157133015

In [77]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

23/08/06 18:24:03 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [78]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [79]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                365|
|   mean|0.13010868614764187|
| stddev|  10.35369958069559|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [80]:
test_data.describe().show()


+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               136|
|   mean|0.5971443343808905|
| stddev| 10.25138938858911|
|    min|-23.51088409032297|
|    max| 23.52945433069272|
+-------+------------------+



In [81]:
correct_model = lr.fit(train_data)

23/08/06 18:24:04 WARN Instrumentation: [19c02533] regParam is zero, which might cause numerical instability and overfitting.


In [82]:
test_results = correct_model.evaluate(test_data)

In [83]:
test_results.rootMeanSquaredError

10.351241793354593